# Data Cleaning

In [27]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# Import custom functions from utils file
from utils import *

# Set display options for data exploratory analysis
pd.set_option('display.max_columns', None)


In [28]:
# Read data
df = pd.read_csv('churn_initial_df.csv')

# Data cleansing & Outlier detection
In this section, we will cover the topic of outliers and data cleansing, as well as inspecting missing values. Outliers are data points that deviate significantly from the rest of the data and can skew analysis, while missing values can introduce bias and inaccuracies in data analysis. We will explore different methods for identifying and handling outliers and missing values, including replacing missing values and clamping outliers. These techniques will help to ensure that our data is clean and reliable for further analysis.

In [29]:
# Inspect the columns, potential null values and data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [30]:
# Print all unique values for each column.
#   This will help us inspect strange values in columns.
for column in df.columns:
    unique_values = df[column].unique()
    print(f"{column}: {unique_values}")

customerID: ['7590-VHVEG' '5575-GNVDE' '3668-QPYBK' ... '4801-JZAZL' '8361-LTMKD'
 '3186-AJIEK']
gender: ['Female' 'Male']
SeniorCitizen: [0 1]
Partner: ['Yes' 'No']
Dependents: ['No' 'Yes']
tenure: [ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26  0
 39]
PhoneService: ['No' 'Yes']
MultipleLines: ['No phone service' 'No' 'Yes']
InternetService: ['DSL' 'Fiber optic' 'No']
OnlineSecurity: ['No' 'Yes' 'No internet service']
OnlineBackup: ['Yes' 'No' 'No internet service']
DeviceProtection: ['No' 'Yes' 'No internet service']
TechSupport: ['No' 'Yes' 'No internet service']
StreamingTV: ['No' 'Yes' 'No internet service']
StreamingMovies: ['No' 'Yes' 'No internet service']
Contract: ['Month-to-month' 'One year' 'Two year']
PaperlessBilling: ['Yes' 'No']
PaymentMethod: ['Electronic check' 'Mailed check' 'Bank transfer (automatic)

### Correct values in column
If the column is not properly classified as boolean, it can affect the performance of certain models or analyses that require boolean data. Additionally, it may be misleading to interpret non-boolean columns as boolean, which can lead to incorrect conclusions or actions based on the data.

In [31]:
# There are some columns that are supposed to have categorical values only,
#   but include more. We need to correct them.

# replace 'No phone service' with NaN values
df['MultipleLines'].replace('No phone service', np.nan, inplace=True)

# replace 'No internet service' with NaN values
df.replace('No internet service', np.nan, inplace=True)

# create a new column 'Has_Internet_Service' which has a value of 1 if any of the columns in a row have a non-NaN value
df['Has_Internet_Service'] = df[['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']].notnull().any(axis=1).astype(int)

# replace 'Yes' with 1 and 'No' with 0
df.replace(['Yes', 'No'], [1, 0], inplace=True)

# fill NaN values with 0s for the new columns
df[['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'MultipleLines']] = df[['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'MultipleLines']].fillna(0).astype('int64')


# Replace Yes, No with 1,0 in all columns where there is
#    only Yes, No values

# Loop through each column
for col in df.columns:
    # Check if column has only 'Yes' and 'No' values
    if set(df[col].unique()) == set(['Yes', 'No']):
        # Replace 'Yes' with 1 and 'No' with 0
        df[col] = df[col].map({'Yes': 1, 'No': 0})


# Define list of columns to convert to numeric
cols_to_convert = ['TotalCharges']

# Loop through each column and convert to float
for col in cols_to_convert:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Convert categorical variables to binary using one-hot encoding
#    The resulting dummy variables are added to the DataFrame as new columns
df = pd.get_dummies(df, columns=["gender", "Contract", "InternetService", "PaymentMethod"])

# Drop the column because it is redundant with Has_Internet_Service column
df.drop('InternetService_0', axis=1, inplace=True)

# Print data types of all columns
print(df.dtypes)

customerID                                  object
SeniorCitizen                                int64
Partner                                      int64
Dependents                                   int64
tenure                                       int64
PhoneService                                 int64
MultipleLines                                int64
OnlineSecurity                               int64
OnlineBackup                                 int64
DeviceProtection                             int64
TechSupport                                  int64
StreamingTV                                  int64
StreamingMovies                              int64
PaperlessBilling                             int64
MonthlyCharges                             float64
TotalCharges                               float64
Churn                                        int64
Has_Internet_Service                         int64
gender_Female                                uint8
gender_Male                    

### Identify duplicates
Duplicate rows can skew summary statistics, such as means and standard deviations, and create biases in machine learning models. They can also cause data redundancy and increase the computational time required to process the data. Therefore, identifying and removing duplicates can improve the quality and reliability of the data used for analysis and modeling.

In [32]:
# Apply function
check_for_duplicates(df)

- There are no duplicated values in the dataframe.


### Missing values
If there are missing values in a dataset, it can impact the statistical power of your analysis, and can even result in incorrect conclusions. Furthermore, many machine learning algorithms require complete data, and missing values can cause errors in these algorithms. Therefore, it is essential to identify and handle missing data appropriately.

In [33]:
# Apply function
check_for_missing_values(df)

- There are missing values in the dataframe. Column 'TotalCharges' has missing values.


In [34]:
# Based on the previous output, there are missing values in the column 'TotalCharges'
#    Therefore, we have to replace NaN values in TotalCharges with 0s
df['TotalCharges'] = df['TotalCharges'].fillna(0).astype('int64')


### Imbalanced dataset
If a dataset is imbalanced, meaning that one class of the target variable is overrepresented compared to the other, the model may learn to predict the majority class more accurately and perform poorly on the minority class. This can result in biased or inaccurate predictions, especially if the minority class is of particular interest. By identifying and addressing class imbalance, such as through resampling techniques or adjusting class weights, a model can be trained to better handle imbalanced data and make more accurate predictions.


In [35]:
# Apply function
check_for_imbalanced_dataset(df)

- The dataframe is imbalanced, with an imbalance ratio of 0.27.



### Outliers
Flagging outliers in a dataset can be useful in identifying potential errors or anomalies in the data, Additionally, you can use the flags as a feature in your model to improve its accuracy, or use them to create new features that capture the presence of outliers in the data. In this function, outliers are detected by comparing each value to the mean and standard deviation of the column, as well as the 1st and 99th percentiles of the column. The function skips outlier detection for binary columns, and joins the outlier flags DataFrame with the input DataFrame to return a new DataFrame with outlier flags. This function can be useful for identifying and handling outliers before training a model.


In [36]:
# Apply function
df = flag_outliers(df)

Outlier column created: tenure
Outlier column created: MonthlyCharges
Outlier column created: TotalCharges


### Clamp outliers
Clamping outliers in numerical columns can help to improve the overall quality of the data and prevent the model from being biased by the presence of extreme values. By replacing outliers with the nearest in-range values, the distribution of the data becomes more normalized, which can help the model make more accurate predictions. Clamping outliers also helps to avoid the risk of overfitting to the training data, which can happen when the model is trying to learn from extreme and non-representative values.

In [37]:
# Apply function
df = clamp_outliers(df)

Columns to be modified: 
tenure
MonthlyCharges
TotalCharges

86 customer IDs affected.


### Optional
It is generally a good practice to scale or normalize your data before using it for model training with XGBoost. The main reason for doing this is to ensure that all features contribute equally to the model and to avoid the domination of certain features over others.

For example, if you do not scale or normalize the TotalCharges feature, it may dominate over other features that have smaller values, causing the model to become biased towards it. By scaling or normalizing the features, you can bring them to the same scale and make them equally important for the model.

Therefore, it is recommended to scale or normalize your data before using it for model training with XGBoost. There are several methods for scaling or normalization, including StandardScaler, MinMaxScaler, and RobustScaler. You can choose the one that best suits your data and the specific problem you are trying to solve.

In [38]:
# # Define the columns to normalize
# # cols_to_normalize = ['MonthlyCharges', 'TotalCharges']
# cols_to_normalize = ['tenure', 'MonthlyCharges', 'TotalCharges']

# # Create the MinMaxScaler object
# scaler = MinMaxScaler()

# # Fit and transform the data
# df[cols_to_normalize] = scaler.fit_transform(df[cols_to_normalize])

# # Check how the column values look after the normalization
# df[cols_to_normalize].head()


### Storing clean dataframe
Store the dataframe after all our operations for modelling

In [40]:
# Store cleaned df for modelling
df.to_csv('churn_cleaned_df.csv', index=False)